# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

# --- UTILIDADES Y MAPPING ---

# Mapeo para convertir la clase CSS del rating (ej: 'Four') a un valor numérico (4)
RATING_MAP = {'One': 1,'Two': 2,'Three': 3,'Four': 4,'Five': 5}

# URL base, utilizando el patrón de paginación confirmado
BASE_URL = "https://books.toscrape.com/catalogue/" 

def extraer_contenido(url):
        response = requests.get(url, timeout=10)
        soup =  BeautifulSoup(response.content, "html.parser")
        return soup


def scrape_books(min_rating: int, max_price: float) -> pd.DataFrame:
    
    all_books_data = []
    page_num = 1

    while True:
        
        current_url = f"{BASE_URL}page-{page_num}.html"
        
        soup = extraer_contenido(current_url)
        
        if soup is None or not soup.find("article", class_="product_pod"):
            break 
        
        articulos = soup.find_all("article", class_="product_pod")

        for art in articulos:
            book_info = {}
            
            try:
                title_tag = art.find("h3").find("a")
                book_info['Title'] = title_tag.get("title", 'Título no encontrado')
                relative_url = title_tag.get("href", '')
                detail_url = BASE_URL + relative_url.replace('../', '')

                price_text = art.find("p", class_="price_color").text
                book_info['Price (£)'] = float(price_text.replace("£", ""))
                
                rating_class = art.find("p", class_=re.compile(r'star-rating'))['class']
                rating_text = rating_class[1] 
                book_info['Rating'] = RATING_MAP.get(rating_text, 0)
                
                availability_text = art.find("p", class_="instock availability").text.strip()
                book_info['Availability'] = availability_text.replace('\n', '').strip()
                
                current_price = book_info.get('Price (£)', 9999.0)
                current_rating = book_info.get('Rating', 0)
                
                if current_rating < min_rating or current_price > max_price:
                    continue 

                soup_detail = extraer_contenido(detail_url)
                
                if soup_detail:
                    upc_td = soup_detail.find('th', string='UPC').find_next_sibling('td')
                    book_info['UPC'] = upc_td.text.strip()
                    
                    breadcrumb = soup_detail.find('ul', class_='breadcrumb').find_all('li')
                    book_info['Genre'] = breadcrumb[2].text.strip()
                        
                    desc_div = soup_detail.find(id='product_description')
                    book_info['Description'] = desc_div.find_next_sibling('p').text.strip()
                else:
                    book_info.update({'UPC': 'N/A', 'Genre': 'N/A', 'Description': 'No disponible'})

                all_books_data.append(book_info)
            
            except Exception as e:
                title = book_info.get('Title', 'Libro Desconocido')
                continue 

        page_num += 1

    df = pd.DataFrame(all_books_data)
    print(f"\n--- Scraping Completado. Total de libros en DataFrame: {len(df)} ---")
    return df

# Parámetros:
MIN_RATING = 4
MAX_PRICE = 20.0

# Llamada a la función
df_resultado = scrape_books(MIN_RATING, MAX_PRICE)

# Mostrar el resultado final
print("\n--- DataFrame Final (libros con Rating ≥ 4 y Precio ≤ £20) ---")
print(df_resultado)


--- Scraping Completado. Total de libros en DataFrame: 75 ---

--- DataFrame Final (libros con Rating ≥ 4 y Precio ≤ £20) ---
                                                Title  Price (£)  Rating  \
0                                         Set Me Free      17.46       5   
1   The Four Agreements: A Practical Guide to Pers...      17.66       5   
2                                      Sophie's World      15.94       5   
3             Untitled Collection: Sabbath Poems 2014      14.27       4   
4                                     This One Summer      19.49       4   
..                                                ...        ...     ...   
70                                    The Zombie Room      19.69       5   
71                                    The Silent Wife      12.34       5   
72                                  The Girl You Lost      12.29       5   
73              The Edge of Reason (Bridget Jones #2)      19.18       4   
74  A Spy's Devotion (The Regency Spi